# Web scraping Encuestas de Hogares

El código scrapea la página web del catálogo de encuestas IHSN y guarda en un excel cada encuesta (posterior o igual a 2015) con su año, país y fuente.
En este ejemplo se limitó el ejercicio a 10 páginas para evitar demoras en la ejecución.

In [117]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [118]:
# Crear un DataFrame vacío
hoja = pd.DataFrame(columns=['Encuesta', 'País', 'Fuente'])

In [119]:
# Obtener las encuestas de las 10 primeras páginas
for pagina in range(1, 10):
    page = requests.get(f'http://catalog.ihsn.org/catalog/?page={pagina}&ps=15')
    soup = BeautifulSoup(page.text, 'html.parser')
    encuestas = soup.find_all('div', {'class': 'col-md-12'})
    
    for encuesta in encuestas:
        Encuesta = encuesta.find('h5', {'class': 'wb-card-title title'})
        País = encuesta.find('div', {'class': 'study-country'})
        Fuente = encuesta.find('div', {'class': 'sub-title'})

        if Encuesta and País and Fuente:
            hoja = pd.concat([hoja, pd.DataFrame({'Encuesta': [Encuesta.text.strip()], 'País': [País.text.strip().rstrip(',')], 'Fuente': [Fuente.text.strip()]})], ignore_index=True)

In [120]:
# Separar la columna "País" en "País" y "Año"
hoja[['País', 'Año']] = hoja['País'].str.split('     ', n=1, expand=True)

# Eliminar espacios de la columna "Año"
hoja['Año'] = hoja['Año'].str.replace(' ', '')

# Eliminar coma al final del valor de la columna "País"
hoja['País'] = hoja['País'].str.rstrip(',')

#Filtrar los años posteriores o iguales a 2015
hoja = hoja[hoja['Año'].str.contains('2015|2016|2017|2018|2019|2020|2021|2022|2023', na=False)]

# Eliminar duplicados
hoja.drop_duplicates(inplace=True)

# Reordenar las columnas
hoja = hoja[['Encuesta', 'País', 'Año', 'Fuente']]

# Ordenar casos por País, luego Año y luego Encuesta
hoja = hoja.sort_values(['País', 'Año', 'Encuesta'])

In [121]:
# Guardar el DataFrame en un Excel
hoja.to_excel('encuestas.xlsx', index=False)